## Calculating Transfer Function between two TE accelerometers with a labJack T7-Pro
This is an attempt to calculate the transfer function from the stored\
accelerations of two accelerometers.  It estimates the TF as the ratio of the\
cross spectral densities H(k) = Pyy(k) / Pxy(k), as in this link:\
https://www.mathworks.com/help/signal/ref/tfestimate.html#bufqg8e

Craig Lage - Sep 27, 2021


In [ ]:
import sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft, fftfreq, ifft, irfft
from scipy.interpolate import splprep, splrep, splev
from scipy.signal import TransferFunction as tf
from scipy.signal import dlsim, csd, welch
from scipy.optimize import minimize

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
# Read in the pickled accelerometer data
scanRate = 1000
readTime = 10.0
df = pd.read_pickle("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6.pkl")

In [ ]:
# Plot the data at different time magnifications.
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)

for n, scanIndex in enumerate([[0, int(scanRate*readTime-1)], [450, 750], [500,600]]):

    sub_df = df[df.index[scanIndex[0]] : df.index[scanIndex[1]]]
    
    plt.subplot(2,3,n+1)
    ax1 = sub_df['a1z'].plot(label="a1z", color='red')
    ax1.set_title("Axis 1", fontsize=16)
    ax1.set_ylabel("Acceleration(g)")
    ax1.legend(loc='center left')

    plt.subplot(2,3,n+4)
    ax2 = sub_df['a2z'].plot(label="a2z", color='red')
    ax2.set_title("Axis 2", fontsize=16)
    ax2.set_ylabel("Acceleration(g)")
    ax2.legend(loc='center left')
    
    if n == 0: # save this data for the FFT
        a1z = np.array(sub_df['a1z'].tolist())
        a2z = np.array(sub_df['a2z'].tolist())

#plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6_14Sep21.pdf")

In [ ]:
# Estimate the TF as  H(k) = Pyy(k) / Pxy(k)  as discussed above
# Plot it several ways

Pyy = csd(a2z, a2z, fs=1000)
Pxy = csd(a1z, a2z, fs=1000)
tf =  Pyy[1] / Pxy[1] # H2 Pyy / Pxy
plt.figure(figsize=(12,3))
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,4,1)
plt.title("Real(TF)", fontsize=24)
plt.plot(Pyy[0], np.real(tf), marker='x', color='blue', label = 'TF')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,2)
plt.title("Imag(TF)", fontsize=24)
plt.plot(Pyy[0], np.imag(tf), marker='x', color='blue', label = 'TF')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,3)
plt.title("Mag(TF)", fontsize=24)
plt.plot(Pyy[0], np.abs(tf), marker='x', color='blue', label = 'TF')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,4)
plt.title("Phase(TF)", fontsize=24)
plt.plot(Pyy[0], np.angle(tf, deg=True), marker='x', color='blue', label = 'TF')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)

## Estimating a functional form for the transfer function.
The next step is to try and find a functional form for the transfer function in a form like this:\
$ H(s) = \frac{a2 s^2 + a1 s + a0}{b2 s^2 + b1 s + b0} $\
Once this is done, scipy routines like scipy.signal.dlsim and scipy.signal.TransferFunction can be used.\
I've done this manually in the cell below, and found a form that fits pretty well.\
However, the a coefficients are complex, and in the scipy routines they need to be real.\
At this point I'm stumped as to how to proceed further.


In [ ]:
xp = 19.0 - Pyy[0] / 5.0
H = (1.8 * xp + 3.0 * 1j) / (1.0 + xp * xp / 3.0)
plt.figure(figsize=(12,3))
plt.subplot(1,4,1)
plt.title("Real(TF)", fontsize=24)
plt.plot(Pyy[0], np.real(tf), marker='x', color='blue', label = 'TF')
plt.plot(Pyy[0], np.real(H), color = 'red', lw = 2.0, label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,2)
plt.title("Imag(TF)", fontsize=24)
plt.plot(Pyy[0], np.imag(tf), marker='x', color='blue', label = 'TF')
plt.plot(Pyy[0], np.imag(H), color = 'red', lw = 2.0, label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,3)
plt.title("Mag(TF)", fontsize=24)
plt.plot(Pyy[0], np.abs(tf), marker='x', color='blue', label = 'TF')
plt.plot(Pyy[0], np.abs(H), color = 'red', lw = 2.0, label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,4)
plt.title("Phase(TF)", fontsize=24)
plt.plot(Pyy[0], np.angle(tf, deg=True), marker='x', color='blue', label = 'TF')
plt.plot(Pyy[0], np.angle(H, deg=True), color = 'red', lw = 2.0, label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)